# imports

Demo account:  
tnfx server  
  
Login:
114116 

Password:
Baraccobbaldo94 

Pass Investitore:
Baraccobbaldo94 


In [1]:
# Trader API
import MetaTrader5 as mt5

# Data manipulation libraries
import pandas as pd
import numpy as np

# Machine learning libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import xgboost as xgb
from scipy.stats import uniform, loguniform

# Technical indicator library
import talib as ta

# Data import library
import yfinance as yf

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import plotly.graph_objs as go

from datetime import datetime
from itertools import count
import time

In [2]:
# connect to the trade account without specifying a password and a server
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
account_number = 114116
authorized = mt5.login(account_number, server='TiranForex-Demo', password='Baraccobbaldo94')
if authorized:
    print(f'connected to account #{account_number}')
else:
    print(f'failed to connect at account #{account_number}, error code: {mt5.last_error()}')

connected to account #114116


In [4]:
BUY = mt5.ORDER_TYPE_BUY
SELL = mt5.ORDER_TYPE_SELL
ORDER_TYPE = BUY

# Downloading stock data

In [62]:
def download_data(stock_indexes):
    '''
    ---
    input: stock indexes list
    output: list of dataframes
    downloads data of stock_indexes and returns list of dataframes, one for each index selected in stock_indexes
    ---
    '''
    df_list = []
    #Download live data
    for index in stock_indexes:
        df_list.append(yf.download(index, period = '1d', interval = '1m'))
    
    return df_list

# Feature engineering

In [63]:
def feature_engineering(df_list):
    '''
    ---
    input: list of dataframes of stock markets data
    output: none
    Performs feature engineering on previously downloaded data, directly modifies items in list
    ---
    '''
    if type(df_list) == list:
        for df in df_list:
            # Drop the rows with zero volume traded
            #df.drop(df[df['Volume'] == 0].index, inplace=True)
            # Create a variable n with a value of 10
            n = 10
            # Create a column by name, RSI and assign the calculation of RSI to it
            df['RSI'] = ta.RSI(np.array(df['Close'].shift(1)), timeperiod=n)

            # Create a column by name, SMA and assign the SMA calculation to it
            df['SMA'] = df['Close'].shift(1).rolling(window=n).mean()

            # Create a column by name, Corr and assign the calculation of correlation to it
            df['Corr'] = df['Close'].shift(1).rolling(window=n).corr(df['SMA'].shift(1))

            # Create a column by name, SAR and assign the SAR calculation to it
            df['SAR'] = ta.SAR(np.array(df['High'].shift(1)), np.array(df['Low'].shift(1)),
                               0.2, 0.2)

            # Create a column by name, ADX and assign the ADX calculation to it
            df['ADX'] = ta.ADX(np.array(df['High'].shift(1)), np.array(df['Low'].shift(1)),
                               np.array(df['Open']), timeperiod=n)

            # Create columns high, low and close with previous minute's OHLC data
            df['Prev_High'] = df['High'].shift(1)
            df['Prev_Low'] = df['Low'].shift(1)
            df['Prev_Close'] = df['Close'].shift(1)

            # Create columns 'OO' with the difference between the current minute's open and last minute's open
            df['OO'] = df['Open']-df['Open'].shift(1)

            # Create columns 'OC' with the difference between the current minute's open and last minute's close
            df['OC'] = df['Open']-df['Prev_Close']

            # Create a column 'Ret' with the calculation of returns
            df['Ret'] = (df['Open'].shift(-1)-df['Open'])/df['Open']

            # Create n columns and assign
            for i in range(1, n):
                df['return%i' % i] = df['Ret'].shift(i)

            # Change the value of 'Corr' to -1 if it is less than -1
            df.loc[df['Corr'] < -1, 'Corr'] = -1

            # Change the value of 'Corr' to 1 if it is greater than 1
            df.loc[df['Corr'] > 1, 'Corr'] = 1


            # Drop the NaN values
            df.dropna(inplace=True)
    else:
        print('ERROR: wrong data type for input, it must be a list of dfs')

# Creating signal target variable

In [64]:
def create_target_variable(df_list):
    '''
    ---
    input: list of dataframes of stock markets data
    output: none
    Modifies dfs in input list to add Signal (target) variable
    ---
    '''
    if type(df_list) == list:
        
        for df in df_list:

            # Create a column by name, 'Signal' and initialize with 0
            df['Signal'] = 0

            # Assign a value of 1 to 'Signal' column for the quantile with the highest returns
            df.loc[df['Ret'] > df['Ret'].quantile(q=0.66), 'Signal'] = 1

            # Assign a value of -1 to 'Signal' column for the quantile with the lowest returns
            df.loc[df['Ret'] < df['Ret'].quantile(q=0.34), 'Signal'] = -1
    else:
        print('ERROR, wrong input type, must be list of dfs')

# Tuning&Training the models

In [65]:
def tune_train_predict(df_list):
    '''
    input: list of dfs
    output: none
    Overwrites list of dfs with pred signal
    '''
    
    
    for df in df_list:
        # Use drop method to drop the columns
        X = df.drop(['Close', 'Adj Close', 'Signal', 'High',
                     'Low', 'Volume', 'Ret'], axis=1)

        # Create a variable which contains all the 'Signal' values
        y = df['Signal']

        ## SVC hyperparams
        # Test variables for 'c' and 'g'
        c = [10, 100, 1000, 10000]
        g = [1e-2, 1e-1, 1e0]

        # Intialize the parameters
        svc_parameters = {'svc__C': c,
                      'svc__gamma': g,
                      'svc__kernel': ['rbf']
                      }

        # Create the 'steps' variable with the pipeline functions
        svc_steps = [('scaler', StandardScaler()), ('svc', SVC())]

        # Pass the 'steps' to the Pipeline function
        svc_pipeline = Pipeline(svc_steps)

        # Call the RandomizedSearchCV function and pass the parameters
        svc_rcv = RandomizedSearchCV(svc_pipeline, svc_parameters, cv=TimeSeriesSplit(n_splits=2))

        # Call the 'fit' method of rcv and pass the train data to it
        svc_rcv.fit(X.iloc[:-1], y.iloc[:-1])

        # Call the 'best_params_' method to obtain the best parameters of C
        best_C = svc_rcv.best_params_['svc__C']

        # Call the 'best_params_' method to obtain the best parameters of kernel
        best_kernel = svc_rcv.best_params_['svc__kernel']

        # Call the 'best_params_' method to obtain the best parameters of gamma
        best_gamma = svc_rcv.best_params_['svc__gamma']

        # Create a new SVC classifier
        cls = SVC(C=best_C, kernel=best_kernel, gamma=best_gamma)

        # Instantiate the StandardScaler
        ss1 = StandardScaler()

        # Pass the scaled train data to the SVC classifier
        cls.fit(ss1.fit_transform(X.iloc[:-1]), y.iloc[:-1])

        # Initiate a column by name, 'Pred_Signal' and assign 0 to it
        df['SVC_Pred_Signal'] = 0

        # Save the predicted values for the train data
        df.iloc[:-1, df.columns.get_loc('SVC_Pred_Signal')] = pd.Series(
            cls.predict(ss1.transform(X.iloc[:-1])).tolist())
        
        # Predicts next step
        df.iloc[-1, df.columns.get_loc('SVC_Pred_Signal')] = pd.Series(
            cls.predict(ss1.transform(X.iloc[len(X)-1:len(X)])).tolist())

In [66]:
if __name__ == '__main__':
    
    #tempo_precedente = int(0)
    evaluation_df_list = []
    i = 0
    minute = 0
    while True:
        #stock_indexes = ['TSLA', 'GOOGL', 'AAPL', 'NFLX', 'NVDA', 'SPY']
        #stock_indexes = ['SIE.DE', ]
        #df_list = download_data(stock_indexes)
        live_hour = time.localtime()[3]
        live_minute = time.localtime()[4]
        live_second = time.localtime()[5]
        
        
        
        if live_minute>minute and live_second>10:
            minute = live_minute
            df_list = []
            trial_df  = yf.download('TSLA', period = '1d', interval = '1m')
            df_list.append(trial_df)
            display(df_list[0])
            #stock_indexes = ['TSLA', 'GOOGL', 'AAPL', 'NFLX', 'NVDA', 'SPY']
            #stock_indexes = ['SIE.DE', ]
            #df_list = download_data(stock_indexes)
            
            i+=1
            print(f'Entering new bid session. {i}th iteration')       
            
            feature_engineering(df_list)
            create_target_variable(df_list)
            tune_train_predict(df_list)
            
            evaluation_df_list.append(df_list[0].iloc[-1])
            print('Exiting bid session')
            
#results_df = pd.concat([df for df in evaluation_df_list], axis=1).T

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-26 09:30:00-05:00,1099.469971,1105.989990,1096.795044,1105.869995,1105.869995,1028124
2021-11-26 09:31:00-05:00,1104.729980,1106.409912,1102.520020,1105.859985,1105.859985,88371
2021-11-26 09:32:00-05:00,1105.859985,1108.782715,1103.000000,1104.989990,1104.989990,118431
2021-11-26 09:33:00-05:00,1098.500000,1098.500000,1097.759155,1097.759155,1097.759155,105134
2021-11-26 09:34:00-05:00,1096.000000,1096.309937,1094.360962,1095.150024,1095.150024,111987
...,...,...,...,...,...,...
2021-11-26 12:39:00-05:00,1092.930054,1092.959961,1088.270020,1088.689941,1088.689941,66740
2021-11-26 12:40:00-05:00,1088.425049,1088.579956,1085.430054,1085.790039,1085.790039,74985
2021-11-26 12:41:00-05:00,1085.670044,1087.469971,1085.500000,1086.790039,1086.790039,40219


Entering new bid session. 1th iteration
Exiting bid session
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-26 09:30:00-05:00,1099.469971,1105.989990,1096.795044,1105.869995,1105.869995,1028124
2021-11-26 09:31:00-05:00,1104.729980,1106.409912,1102.520020,1105.859985,1105.859985,88371
2021-11-26 09:32:00-05:00,1105.859985,1108.782715,1103.000000,1104.989990,1104.989990,118431
2021-11-26 09:33:00-05:00,1098.500000,1098.500000,1097.759155,1097.759155,1097.759155,105134
2021-11-26 09:34:00-05:00,1096.000000,1096.309937,1094.360962,1095.150024,1095.150024,111987
...,...,...,...,...,...,...
2021-11-26 12:40:00-05:00,1088.425049,1088.579956,1085.430054,1085.790039,1085.790039,74985
2021-11-26 12:41:00-05:00,1085.670044,1087.469971,1085.500000,1086.790039,1086.790039,40219
2021-11-26 12:42:00-05:00,1087.089844,1087.089844,1085.277954,1085.772217,1085.772217,45967


Entering new bid session. 2th iteration
Exiting bid session
[*********************100%***********************]  1 of 1 completed


...

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-26 09:30:00-05:00,1099.469971,1105.989990,1096.795044,1105.869995,1105.869995,1028124
2021-11-26 09:31:00-05:00,1104.729980,1106.409912,1102.520020,1105.859985,1105.859985,88371
2021-11-26 09:32:00-05:00,1105.859985,1108.782715,1103.000000,1104.989990,1104.989990,118431
2021-11-26 09:33:00-05:00,1098.500000,1098.500000,1097.759155,1097.759155,1097.759155,105134
2021-11-26 09:34:00-05:00,1096.000000,1096.309937,1094.360962,1095.150024,1095.150024,111987
...,...,...,...,...,...,...
2021-11-26 12:54:00-05:00,1082.800049,1083.640015,1081.000000,1081.369995,1081.369995,106845
2021-11-26 12:55:00-05:00,1081.380005,1083.500000,1081.000000,1083.290039,1083.290039,102973
2021-11-26 12:56:00-05:00,1083.030029,1085.502075,1082.900024,1084.640015,1084.640015,79990


Entering new bid session. 16th iteration
Exiting bid session
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-26 09:30:00-05:00,1099.469971,1105.989990,1096.795044,1105.869995,1105.869995,1028124
2021-11-26 09:31:00-05:00,1104.729980,1106.409912,1102.520020,1105.859985,1105.859985,88371
2021-11-26 09:32:00-05:00,1105.859985,1108.782715,1103.000000,1104.989990,1104.989990,118431
2021-11-26 09:33:00-05:00,1098.500000,1098.500000,1097.759155,1097.759155,1097.759155,105134
2021-11-26 09:34:00-05:00,1096.000000,1096.309937,1094.360962,1095.150024,1095.150024,111987
...,...,...,...,...,...,...
2021-11-26 12:55:00-05:00,1081.380005,1083.500000,1081.000000,1083.290039,1083.290039,102973
2021-11-26 12:56:00-05:00,1083.030029,1085.502075,1082.900024,1084.640015,1084.640015,79990
2021-11-26 12:57:00-05:00,1084.640015,1087.540039,1084.449951,1086.430054,1086.430054,85835


Entering new bid session. 17th iteration
Exiting bid session


KeyboardInterrupt: 

In [80]:
results_df = pd.concat([df for df in evaluation_df_list], axis=1).T
results_df = results_df.drop_duplicates()
results_df

,Open,High,Low,Close,Adj Close,Volume,RSI,SMA,Corr,SAR,...,return2,return3,return4,return5,return6,return7,return8,return9,Signal,SVC_Pred_Signal
2021-11-26 12:42:00-05:00,1087.089844,1087.089844,1085.277954,1085.772217,1085.772217,45967.0,35.397267,1089.844934,0.299567,1092.959961,...,-0.002531,-0.004122,0.001283,-0.000220,0.000926,-0.000014,0.001372,0.000085,-1.0,1.0
2021-11-26 12:43:00-05:00,1085.900024,1086.750000,1085.170044,1085.170044,1085.170044,37887.0,32.679977,1089.497156,0.519866,1091.453979,...,0.001308,-0.002531,-0.004122,0.001283,-0.000220,0.000926,-0.000014,0.001372,-1.0,-1.0
2021-11-26 12:44:00-05:00,1085.109985,1086.109985,1084.370117,1085.661255,1085.661255,60322.0,31.110069,1089.065527,0.658019,1090.218774,...,-0.001094,0.001308,-0.002531,-0.004122,0.001283,-0.000220,0.000926,-0.000014,0.0,0.0
2021-11-26 12:45:00-05:00,1085.823853,1085.879883,1084.689941,1085.303467,1085.303467,30555.0,33.984441,1088.554858,0.704315,1089.209028,...,-0.000728,-0.001094,0.001308,-0.002531,-0.004122,0.001283,-0.000220,0.000926,-1.0,1.0
2021-11-26 12:46:00-05:00,1084.819946,1087.150024,1084.770142,1086.826782,1086.826782,56930.0,32.869859,1087.942200,0.712226,1088.241246,...,0.000658,-0.000728,-0.001094,0.001308,-0.002531,-0.004122,0.001283,-0.000220,1.0,1.0
2021-11-26 12:47:00-05:00,1086.420044,1087.119995,1086.000000,1087.060059,1087.060059,79745.0,41.853731,1087.424878,0.589175,1087.467020,...,-0.000925,0.000658,-0.000728,-0.001094,0.001308,-0.002531,-0.004122,0.001283,1.0,1.0
2021-11-26 12:48:00-05:00,1086.979980,1087.339966,1084.199951,1084.209961,1084.209961,69482.0,43.148307,1086.986890,0.417892,1087.150024,...,0.001475,-0.000925,0.000658,-0.000728,-0.001094,0.001308,-0.002531,-0.004122,-1.0,-1.0
2021-11-26 12:49:00-05:00,1084.209961,1086.000000,1083.510010,1084.199951,1084.199951,110828.0,33.133905,1086.127380,0.428567,1084.199951,...,0.000515,0.001475,-0.000925,0.000658,-0.000728,-0.001094,0.001308,-0.002531,0.0,1.0
2021-11-26 12:50:00-05:00,1084.000000,1084.920044,1083.259888,1084.339966,1084.339966,58111.0,33.103923,1085.678381,0.385138,1087.339966,...,-0.002548,0.000515,0.001475,-0.000925,0.000658,-0.000728,-0.001094,0.001308,0.0,1.0
2021-11-26 12:51:00-05:00,1084.489990,1085.189941,1083.699951,1084.420044,1084.420044,44958.0,34.031676,1085.533374,0.537970,1087.339966,...,-0.000194,-0.002548,0.000515,0.001475,-0.000925,0.000658,-0.000728,-0.001094,0.0,1.0


In [82]:
truth_df = df_list[0].drop_duplicates()
truth_df = truth_df.loc[results_df.index[0]:]
truth_df

,Open,High,Low,Close,Adj Close,Volume,RSI,SMA,Corr,SAR,...,return2,return3,return4,return5,return6,return7,return8,return9,Signal,SVC_Pred_Signal
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-26 12:42:00-05:00,1086.944946,1087.089844,1085.277954,1085.772217,1085.772217,46113,35.391335,1089.844934,0.299567,1092.959961,...,-0.002531,-0.004122,0.001521,-0.000458,0.000637,0.000119,0.001528,0.000085,-1,-1
2021-11-26 12:43:00-05:00,1085.900024,1086.773315,1085.170044,1085.170044,1085.170044,37912,32.674921,1089.497156,0.520591,1091.451978,...,0.001174,-0.002531,-0.004122,0.001521,-0.000458,0.000637,0.000119,0.001528,-1,-1
2021-11-26 12:44:00-05:00,1085.109985,1086.109985,1084.370117,1085.661255,1085.661255,0,31.105488,1089.065527,0.658443,1090.217173,...,-0.000961,0.001174,-0.002531,-0.004122,0.001521,-0.000458,0.000637,0.000119,1,1
2021-11-26 12:45:00-05:00,1085.823853,1085.879883,1084.689941,1085.303467,1085.303467,30555,33.979645,1088.554858,0.704519,1089.207747,...,-0.000728,-0.000961,0.001174,-0.002531,-0.004122,0.001521,-0.000458,0.000637,-1,-1
2021-11-26 12:46:00-05:00,1084.819946,1087.150024,1084.770142,1086.826782,1086.826782,56930,32.869859,1087.942200,0.712226,1088.240221,...,0.000658,-0.000728,-0.000961,0.001174,-0.002531,-0.004122,0.001521,-0.000458,1,1
2021-11-26 12:47:00-05:00,1086.420044,1087.119995,1086.000000,1087.060059,1087.060059,79745,41.853731,1087.424878,0.589175,1087.466200,...,-0.000925,0.000658,-0.000728,-0.000961,0.001174,-0.002531,-0.004122,0.001521,1,1
2021-11-26 12:48:00-05:00,1086.979980,1087.339966,1084.199951,1084.209961,1084.209961,69482,43.148307,1086.986890,0.417892,1087.150024,...,0.001475,-0.000925,0.000658,-0.000728,-0.000961,0.001174,-0.002531,-0.004122,-1,-1
2021-11-26 12:49:00-05:00,1084.209961,1086.000000,1083.510010,1084.199951,1084.199951,110828,33.133905,1086.127380,0.428567,1084.199951,...,0.000515,0.001475,-0.000925,0.000658,-0.000728,-0.000961,0.001174,-0.002531,0,0
2021-11-26 12:50:00-05:00,1084.000000,1084.920044,1083.259888,1084.339966,1084.339966,58111,33.103923,1085.678381,0.385138,1087.339966,...,-0.002548,0.000515,0.001475,-0.000925,0.000658,-0.000728,-0.000961,0.001174,0,0


In [83]:
truth_df['Pred_signal']=results_df['SVC_Pred_Signal']
truth_df['SVC_Strat_Ret'] = truth_df['Ret']*truth_df['Pred_signal']
truth_df

,Open,High,Low,Close,Adj Close,Volume,RSI,SMA,Corr,SAR,...,return4,return5,return6,return7,return8,return9,Signal,SVC_Pred_Signal,Pred_signal,SVC_Strat_Ret
Datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-26 12:42:00-05:00,1086.944946,1087.089844,1085.277954,1085.772217,1085.772217,46113,35.391335,1089.844934,0.299567,1092.959961,...,0.001521,-0.000458,0.000637,0.000119,0.001528,0.000085,-1,-1,1.0,-0.000961
2021-11-26 12:43:00-05:00,1085.900024,1086.773315,1085.170044,1085.170044,1085.170044,37912,32.674921,1089.497156,0.520591,1091.451978,...,-0.004122,0.001521,-0.000458,0.000637,0.000119,0.001528,-1,-1,-1.0,0.000728
2021-11-26 12:44:00-05:00,1085.109985,1086.109985,1084.370117,1085.661255,1085.661255,0,31.105488,1089.065527,0.658443,1090.217173,...,-0.002531,-0.004122,0.001521,-0.000458,0.000637,0.000119,1,1,0.0,0.000000
2021-11-26 12:45:00-05:00,1085.823853,1085.879883,1084.689941,1085.303467,1085.303467,30555,33.979645,1088.554858,0.704519,1089.207747,...,0.001174,-0.002531,-0.004122,0.001521,-0.000458,0.000637,-1,-1,1.0,-0.000925
2021-11-26 12:46:00-05:00,1084.819946,1087.150024,1084.770142,1086.826782,1086.826782,56930,32.869859,1087.942200,0.712226,1088.240221,...,-0.000961,0.001174,-0.002531,-0.004122,0.001521,-0.000458,1,1,1.0,0.001475
2021-11-26 12:47:00-05:00,1086.420044,1087.119995,1086.000000,1087.060059,1087.060059,79745,41.853731,1087.424878,0.589175,1087.466200,...,-0.000728,-0.000961,0.001174,-0.002531,-0.004122,0.001521,1,1,1.0,0.000515
2021-11-26 12:48:00-05:00,1086.979980,1087.339966,1084.199951,1084.209961,1084.209961,69482,43.148307,1086.986890,0.417892,1087.150024,...,0.000658,-0.000728,-0.000961,0.001174,-0.002531,-0.004122,-1,-1,-1.0,0.002548
2021-11-26 12:49:00-05:00,1084.209961,1086.000000,1083.510010,1084.199951,1084.199951,110828,33.133905,1086.127380,0.428567,1084.199951,...,-0.000925,0.000658,-0.000728,-0.000961,0.001174,-0.002531,0,0,1.0,-0.000194
2021-11-26 12:50:00-05:00,1084.000000,1084.920044,1083.259888,1084.339966,1084.339966,58111,33.103923,1085.678381,0.385138,1087.339966,...,0.001475,-0.000925,0.000658,-0.000728,-0.000961,0.001174,0,0,1.0,0.000452


In [84]:
#declare figure
fig2 = go.Figure()

#Set up traces
fig2.add_trace(go.Scatter(x=truth_df.index, y= (truth_df['Ret']+1).cumprod(),line=dict(color='royalblue', width=.8), name = 'stock_returns'))
fig2.add_trace(go.Scatter(x=truth_df.index, y= (truth_df['SVC_Strat_Ret']+1).cumprod(),line=dict(color='orange', width=.8), name = 'SVC_strategy_returns'))

# Add titles
fig2.update_layout(title=' fare cassh o no?',
                   yaxis_title='Stock return (% Return)')
fig2.show()

In [ ]:
if __name__ == '__main__':
    
    tempo_precedente = int(0)
    #evaluation_df_list = []
    while True:
        
        tempo = int(time.time())
        if tempo >= tempo_precedente + 60:
            print('Entering new bid session')
            tempo_precedente = tempo
            
            #stock_indexes = ['TSLA', 'GOOGL', 'AAPL', 'NFLX', 'NVDA', 'SPY']
            #df_list = download_data(stock_indexes)
            trial_df  = yf.download('LMVH', period = '1d', interval = '1m')
            df_list = []
            df_list.append(trial_df)
            
            print(trial_df.iloc[len(trial_df)-1:len(trial_df)].index[0])
            
            feature_engineering(df_list)
            create_target_variable(df_list)
            tune_train_predict(df_list)
            
            #evaluation_df_list.append(df_list[0][['Open', 'Close', 'Signal', 'SVC_Pred_Signal']].iloc[-1])
            print('Exiting bid session')
            
#results_df = pd.concat([df for df in evaluation_df_list], axis=1).T

# Evaluating performance

In [ ]:
returns_dict = dict()

for df, index in zip(df_list, stock_indexes): 
    #declare figure
    fig2 = go.Figure()

    #Set up traces
    fig2.add_trace(go.Scatter(x=df.iloc[split:].index, y= (df['Ret'][split:]+1).cumprod(),line=dict(color='royalblue', width=.8), name = 'stock_returns'))
    fig2.add_trace(go.Scatter(x=df.iloc[split:].index, y= (df['SVC_Strat_Ret'][split:]+1).cumprod(),line=dict(color='orange', width=.8), name = 'SVC_strategy_returns'))

    fig2.add_vline(x=df.iloc[split:split+1].index[0], line=dict(color='darkgreen', dash='dot'))
    
    # Add titles
    fig2.update_layout(title=index + ' fare cassh o no?',
                       yaxis_title='Stock return (% Return)')
    fig2.show()
    
    returns_dict[index] = (df['SVC_Strat_Ret'][split:]+1).cumprod()[-1] -1

In [ ]:
for df, index in zip(df_list, stock_indexes): 
    #declare figure
    fig2 = go.Figure()

    #Set up traces
    fig2.add_trace(go.Scatter(x=df.index, y= (df['Ret']+1).cumprod(),line=dict(color='royalblue', width=.8), name = 'stock_returns'))
    fig2.add_trace(go.Scatter(x=df.index, y= (df['SVC_Strat_Ret']+1).cumprod(),line=dict(color='orange', width=.8), name = 'SVC_strategy_returns'))

    fig2.add_vline(x=df.iloc[split:split+1].index[0], line=dict(color='darkgreen', dash='dot'))

    #fig.add_shape()
    # Add titles
    fig2.update_layout(title=index + ' fare cassh o no?',
                       yaxis_title='Stock return (% Return)')
    fig2.show()

In [ ]:
returns_dict

In [ ]:
i = 0
for value in returns_dict.values():
    i += value
i = i/len(returns_dict)
print(i)

**Now is time to link the decisions to a trading bot**  
*Metatrading5 - tutorial @ https://www.cryptomaton.org/2021/03/14/how-to-code-your-own-crypto-trading-bot-python/*

**Notes on algorithm and future work:**  
*Notes:*  
For now seems like the svc strategy is profitable over various US stock indices.  
The strategy though needs 80% of the day trading data. E.g. US market is open from 9:00 to 16:00 (NY time), we will thus need data from 9:00 to 14:33 approx for training the algorithm and we will be trading from 14:33 onwards. Returns shown above are computed from 14:33 onwards (or were the split variable was computed). 
  
I think this means we will have to:  
- modify the split variable used in code above (we wont have all the day data when live training the network, moreover different indexes have different data lenght)
- update algorithm so that it can use stock indexes from European and Asian markets as well in order to maximize trading time in a day

In [ ]:
for df in df_list: print(len(df))